In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.dates as mdates
from sklearn.model_selection import train_test_split
from statsmodels.tsa.api import SimpleExpSmoothing
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, mean_absolute_error

In [2]:
df = pd.read_csv('../../Datasets/NVDA Historical Data.csv')
df

Date   Price    Open    High     Low     Vol. Change %
0      1/3/2017   25.50   26.10   26.59   24.84  150.20M   -4.42%
1      1/4/2017   26.10   25.85   26.38   25.38  119.92M    2.35%
2      1/5/2017   25.43   26.13   26.45   25.26   98.43M   -2.57%
3      1/6/2017   25.77   25.71   26.06   25.30   82.29M    1.34%
4      1/9/2017   26.82   25.88   27.00   25.88   91.62M    4.07%
...         ...     ...     ...     ...     ...      ...      ...
1606  5/22/2023  311.76  309.01  315.20  306.80   36.97M   -0.28%
1607  5/23/2023  306.88  310.00  312.88  306.31   35.47M   -1.57%
1608  5/24/2023  305.38  302.10  306.07  298.06   55.28M   -0.49%
1609  5/25/2023  379.80  385.23  394.80  366.35  153.46M   24.37%
1610  5/26/2023  389.46  378.90  391.70  375.50   71.14M    2.54%

[1611 rows x 7 columns]

In [3]:
df.info

<bound method DataFrame.info of            Date   Price    Open    High     Low     Vol. Change %
0      1/3/2017   25.50   26.10   26.59   24.84  150.20M   -4.42%
1      1/4/2017   26.10   25.85   26.38   25.38  119.92M    2.35%
2      1/5/2017   25.43   26.13   26.45   25.26   98.43M   -2.57%
3      1/6/2017   25.77   25.71   26.06   25.30   82.29M    1.34%
4      1/9/2017   26.82   25.88   27.00   25.88   91.62M    4.07%
...         ...     ...     ...     ...     ...      ...      ...
1606  5/22/2023  311.76  309.01  315.20  306.80   36.97M   -0.28%
1607  5/23/2023  306.88  310.00  312.88  306.31   35.47M   -1.57%
1608  5/24/2023  305.38  302.10  306.07  298.06   55.28M   -0.49%
1609  5/25/2023  379.80  385.23  394.80  366.35  153.46M   24.37%
1610  5/26/2023  389.46  378.90  391.70  375.50   71.14M    2.54%

[1611 rows x 7 columns]>

In [4]:
# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'Date' column as the index value
df.set_index('Date', inplace=True)

In [5]:
print(df.index)

DatetimeIndex(['2017-01-03', '2017-01-04', '2017-01-05', '2017-01-06',
               '2017-01-09', '2017-01-10', '2017-01-11', '2017-01-12',
               '2017-01-13', '2017-01-17',
               ...
               '2023-05-15', '2023-05-16', '2023-05-17', '2023-05-18',
               '2023-05-19', '2023-05-22', '2023-05-23', '2023-05-24',
               '2023-05-25', '2023-05-26'],
              dtype='datetime64[ns]', name='Date', length=1611, freq=None)


In [6]:
df = df[['Price']]
df = df.dropna() # Drop missing values
# df = df.reset_index(drop=True) # Reset the index

# Split the data into training, testing, and validation sets
train_size = int(0.6 * len(df))
test_size = int(0.2 * len(df))
val_size = len(df) - train_size - test_size

train_data = df[:train_size]
test_data = df[train_size:train_size+test_size]
val_data = df[train_size+test_size:]

In [7]:
# Train the SES model and tune alpha using the validation dataset
best_alpha = None
best_mse = np.inf
for alpha in np.arange(0.1, 1.0, 0.1):
    model = SimpleExpSmoothing(train_data['Price']).fit(smoothing_level=alpha, optimized=False)
    y_pred = model.forecast(len(val_data))
    mse = mean_squared_error(val_data['Price'], y_pred)
    if mse < best_mse:
        best_mse = mse
        best_alpha = alpha

print("Best alpha value:", best_alpha)

Best alpha value: 0.1


C:\Users\dinht\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\dinht\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\dinht\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
 

In [8]:
# Training on the train set and testing on the test set
history = train_data.Price.tolist()
y_pred = []
ytest = np.array(test_data['Price'])

for i in test_data.Price:
  model = SimpleExpSmoothing(history)
  model_fit = model.fit(smoothing_level = 0.7)
  prediction = model_fit.forecast()  
  y_pred.append(prediction[0])
  history.append(i)

In [9]:
# Using the train set and the test set for training, then predicting on the validation set
y_pred_val = []
yval = np.array(val_data['Price'])

for i in val_data.Price:
  model = SimpleExpSmoothing(history)
  model_fit = model.fit(smoothing_level = 0.7)
  prediction = model_fit.forecast()  
  y_pred_val.append(prediction[0])
  history.append(i)

In [10]:
# Regression metrics
test_rmse = np.sqrt(np.mean((y_pred - ytest)**2))
test_mse = mean_squared_error(ytest, y_pred)
test_mae = mean_absolute_error(ytest, y_pred)

valid_rmse = np.sqrt(np.mean((y_pred_val - yval)**2))
valid_mse = mean_squared_error(yval, y_pred_val)
valid_mae = mean_absolute_error(yval, y_pred_val)

print('Testing RMSE:', test_rmse)
print('Testing MSE:', test_mse)
print('Testing MAE:', test_mae)

print('Validation RMSE:', valid_rmse)
print('Validation MSE:', valid_rmse)
print('Validation MAE:', valid_mae)

Testing RMSE: 6.452006967173041
Testing MSE: 41.62839390444947
Testing MAE: 4.48618997845081
Validation RMSE: 8.762148533647487
Validation MSE: 8.762148533647487
Validation MAE: 6.341162253545557


In [11]:
# The process of creating index predict next 30 days

# Get the last date from the current index
last_date = df.index[-1]

# Add one day to the last date
next_date = last_date + pd.DateOffset(days=1)

# Create a list of indices starting from the next date for 30 days
index_next_30_days = pd.date_range(start=next_date, periods=30).tolist()

# Predict the closing prices for the next 30 days
y_next_30_days = []

for i in range(30):
  model = SimpleExpSmoothing(history)
  model_fit = model.fit(smoothing_level = best_alpha)
  prediction = model_fit.forecast()  
  y_next_30_days.append(prediction[0])
  history.append(prediction[0])

# Print the predicted closing prices for the next 30 days
print('Predicted prices for the next 30 days:')
print(y_next_30_days)

Predicted prices for the next 30 days:
[311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076, 311.70369454909076]


In [12]:
print(last_date)

2023-05-26 00:00:00


In [13]:
# Drawing
plt.figure(figsize=(25, 10))
plt.plot(df.index[:train_size], train_data)
plt.plot(df.index[train_size : train_size + test_size], test_data)
plt.plot(df.index[train_size : train_size + test_size], y_pred)
plt.plot(df.index[train_size + test_size :], y_pred_val)
plt.plot(df.index[train_size + test_size :], val_data)
plt.plot(index_next_30_days, y_next_30_days)
plt.legend(['Train', 'Test', 'PredictTest', 'Validate', 'PredictValidate', 'Next30Day'])

plt.grid()
# plt.savefig('SES_AMD.png')
plt.show()

In [14]:
test_mse = mean_squared_error(test_data, y_pred)
print(test_mse)

41.62839390444947
